In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import warnings
import os
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
warnings.filterwarnings('ignore')

In [4]:
train = pd.read_parquet("/home/aimlproject1/train_low_mem.parquet")
labels= pd.read_csv('./label_.csv')

In [6]:
def has_duplicate(train):
    for i,g in train.groupby('investment_id'):
        X = np.array(g['time_id'])
        if(len(np.unique(X)) != len(X)):
            return "DUP"
    return "NO"
print(has_duplicate(train))
dict = {}
x=[]
for i,g in tqdm(train.groupby('label_feature')):
    x = (abs(g.drop(['time_id','investment_id','label_feature'],axis=1).corr()['target'])).sort_values(ascending = False)
    best_f = x.index[1:11]
    dict[i] = best_f

NO


 33%|███▎      | 2/6 [00:46<01:19, 19.95s/it]

In [ ]:

for i,g in train.groupby('label_feature'): 
    t = ['investment_id','time_id'  , 'target',*dict[i] ]
    g = g[[*t]].sort_values(by= ["investment_id" , "time_id"])
    g.to_csv('cluster/' + str(i)  + "_.csv" , index=False)

In [ ]:
def printMissing(arr, n, low, high):
    l = [] 
    points_of_range = [False] * (high-low+1)
     
    for i in range(n) :
        if ( low <= arr[i] and arr[i] <= high ) :
            points_of_range[arr[i]-low] = True

    for x in range(high-low+1) :
        if (points_of_range[x]==False) :
            l.append(low+x)
    
    return l 
def imputer(df) :

    arr = df.to_numpy()
    s = arr.mean(axis=0)
    s = s.reshape(1,-1)
    l = printMissing(arr , (len(arr)) , (int)(np.min(arr)) , (int)(np.max(arr)))
    for i in l : 
        s[0][1] = i 
        arr=np.append(arr,s,axis=0)
    # arr.sort_values(by = 'time_id')
    arr = arr[arr[:,1].argsort()]
    return arr 


In [ ]:
files = ["0_" , "1_" , "2_" , "3_" , "-1_" , "4_"]
for i in tqdm(files): 
    PATH  = "./cluster/" + i + ".csv"
    frame = pd.read_csv(PATH) 
    grouped  = frame.groupby('investment_id')
    for i,g in grouped :
        print(i)
        column_names = [str(s) for s in g.columns]
        np_arr = imputer(g)
        df = pd.DataFrame(data = np_arr,  
                  columns = column_names)
        print(has_duplicate(df))
        df.to_csv('imp_mean.csv', mode='a', header=not os.path.exists('imp_mean.csv'))

In [ ]:
r = pd.read_csv('./imp_mean.csv')
r = r.drop('Unnamed: 0' , axis  = 1)

In [ ]:
r.head()


In [ ]:
def NA(df):
    df = df.to_numpy()
    inv_id = df[0][0]
    min_max = ((int)(df.min(axis=0)[1]) , (int)(df.max(axis=0)[1]))

    l=[]
    for i in range(min_max[0],min_max[1]):
        if i not in df[:, 1]:
            l.append(i)
    # print(l)
    arr = np.empty((1,13))
    arr[:] = np.nan
    for i in l:
        arr[0][0] = inv_id
        arr[0][1] = i
        df = np.concatenate((df, arr), axis=0)
    df = df[df[:,1].argsort()]
    return df



files = ["0_" , "1_" , "2_" , "3_" , "-1_" , "4_"]
for i in tqdm(files): 
    PATH  = "./cluster/" + i + ".csv"
    frame = pd.read_csv(PATH) 
    grouped  = frame.groupby('investment_id')
    for i,g in grouped :
        # print(i)
        column_names = [str(s) for s in g.columns]
        np_arr = NA(g)
        df = pd.DataFrame(data = np_arr,columns = column_names)
        iter_imputer = IterativeImputer(random_state=42)
        iter_imputed = iter_imputer.fit_transform(df)
        df = pd.DataFrame(iter_imputed, columns=df.columns)
        df.to_csv('imp_itr.csv', mode='a', header=not os.path.exists('imp_itr.csv'))
        


In [ ]:
count = 0 
r2 = pd.read_csv('./imp_itr.csv')
# r2 = r.drop('Unnamed: 0' , axis  = 1)
for i , g in r2.groupby(['investment_id']):
      arr = np.array(g.time_id , dtype = int)
      count+= len(printMissing(arr , (len(arr)) , (int)(np.min(arr)) , (int)(np.max(arr))))

print(count)    